# s{t} = [X{t}, Y{t}, X{t−1}, Y{t−1},..., X{t−7}, Y{t−7}, C].
The input features s{t} are processed by a residual tower that consists of a single convolutional block 
followed by either 19 or 39 residual blocks.

The convolutional block applies the following modules:
(1) A convolution of 256 filters of kernel size 3 × 3 with stride 1
(2) Batch normalization
(3) A rectifier nonlinearity

Each residual block applies the following modules sequentially to its input: 
(1) A convolution of 256 filters of kernel size 3 × 3 with stride 1
(2) Batch normalization
(3) A rectifier nonlinearity
(4) A convolution of 256 filters of kernel size 3 × 3 with stride 1
(5) Batch normalization
(6) A skip connection that adds the input to the block
(7) A rectifier nonlinearity

The output of the residual tower is passed into two separate ‘heads’ for computing the policy and value. 

The policy head applies the following modules: 
(1) A convolution of 2 filters of kernel size 1 × 1 with stride 1
(2) Batch normalization
(3) A rectifier nonlinearity
(4) A fully connected linear layer that outputs a vector of size 19*19 + 1 = 362, corresponding to logit probabilities for all intersections and the pass move

The value head applies the following modules:
(1) A convolution of 1 filter of kernel size 1 × 1 with stride 1 
(2) Batch normalization
(3) A rectifier nonlinearity
(4) A fully connected linear layer to a hidden layer of size 256
(5) A rectifier nonlinearity
(6) A fully connected linear layer to a scalar
(7) A tanh nonlinearity outputting a scalar in the range [−1, 1]

The overall network depth, in the 20­ or 40­ block network, is 39 or 79 parameterized layers, respectively, 
for the residual tower, plus an additional 2 layers for the policy head and 3 layers for the value head.
"""


In [4]:
import keras
from keras.layers import Activation, BatchNormalization
from keras.layers import Conv2D, Dense, Flatten, Input
from keras.models import Model
import numpy as np
import time



class smallNN:
    def __init__(self):
        #self.board_input = Input(shape=TrojanGoPlane.shape(), name='board_input')
        self.board_input = Input(shape=(7, 5, 5), name='board_input')

    def nn_model(self):

        pb = self.board_input
        for i in range(4):                     # <1>
            pb = Conv2D(64, (3, 3),        # <1>
                padding='same',                # <1>
                data_format='channels_first',  # <1>
                activation='relu')(pb)     # <1>



        policy_conv = \
            Conv2D(2, (1, 1),                          # <2>
                data_format='channels_first',          # <2>
                activation='relu')(pb)                 # <2>
        policy_flat = Flatten()(policy_conv)           # <2>
        policy_output = \
            Dense(26,
                  activation='softmax')(policy_flat)   # <2>




        value_conv = \
            Conv2D(1, (1, 1),                                    # <3>
                data_format='channels_first',                    # <3>
                activation='relu')(pb)                           # <3>
        value_flat = Flatten()(value_conv)                       # <3>
        value_hidden = Dense(256, activation='relu')(value_flat) # <3>
        value_output = Dense(1, activation='tanh')(value_hidden) # <3>

        model = Model(
            inputs=[self.board_input],
            outputs=[policy_output, value_output])

        return model



class trojanGoZero:
    def __init__(self, num_resnet_block=19):
        #self.board_input = Input(shape=TrojanGoPlane.shape(), name='board_input')
        self.board_input = Input(shape=(7, 5, 5), name='board_input')
        self.num_resnet_block = num_resnet_block
        self.num_filters = 256

    
    def resNetBlock(self, x,filters,pool=False):
        res = x
        
        if pool:
            x = MaxPooling2D(pool_size=(2, 2))(x)
            res = Conv2D(filters=filters,kernel_size=[1,1],strides=(2,2),padding="same", data_format='channels_first')(res)
        #out = BatchNormalization()(x)
        #out = Activation("relu")(out)
            
        out = Conv2D(filters=filters, kernel_size=[3, 3], strides=[1, 1], padding="same", data_format='channels_first')(x)
        out = BatchNormalization()(out)
        out = Activation("relu")(out)
        
        out = Conv2D(filters=filters, kernel_size=[3, 3], strides=[1, 1], padding="same", data_format='channels_first')(out)
        out = BatchNormalization()(out)
        
        out = keras.layers.add([res,out])
        out = Activation("relu")(out)

        return out


        
    def nn_model(self, input_shape):
        #Input feature of 17*19*19 or 7*5*5 as board_input or board_images
        
        board_images = Input(input_shape)
        #board_images = self.board_input

        #CNN-1 with Batch Normalization and rectifier nonlinearity.
        cnn1 = Conv2D(filters=256, kernel_size=[3, 3], strides=[1, 1], padding="same", data_format='channels_first')(board_images)
        cnn1_batch = BatchNormalization()(cnn1)
        cnn1_act = Activation("relu")(cnn1_batch)

        self_in = cnn1_act
        
        #Now build 19 or 39 ResNet block networks depends on "num_resnet_block" variable.
        for i in range(self.num_resnet_block):
            self_out = self.resNetBlock(self_in, self.num_filters)
            self_in = self_out

        out = self_out
        
        policy_conv = \
            Conv2D(2, (1, 1),                          # <2>
                data_format='channels_first',          # <2>
                activation='relu')(out)                # <2>
        policy_conv_bn = BatchNormalization()(policy_conv)
        policy_flat = Flatten()(policy_conv_bn)           # <2>
        policy_output = \
            Dense(26,
                  activation='softmax')(policy_flat)   # <2>




        value_conv = \
            Conv2D(1, (1, 1),                                    # <3>
                data_format='channels_first',                    # <3>
                activation='relu')(out)                          # <3>
        value_conv_bn = BatchNormalization()(value_conv)
        value_flat = Flatten()(value_conv_bn)                       # <3>
        value_hidden = Dense(256, activation='relu')(value_flat) # <3>
        value_output = Dense(1, activation='tanh')(value_hidden) # <3>

        model = Model(
            inputs=[board_images],
            outputs=[policy_output, value_output])

        return model


In [5]:
net = trojanGoZero()
input_shape = (7,5,5)
model = net.nn_model(input_shape)
print(model.summary())
    
  

net_small = smallNN()
model_small = net_small.nn_model()
print(model_small.summary())
    

Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 7, 5, 5)      0                                            
__________________________________________________________________________________________________
conv2d_95 (Conv2D)              (None, 256, 5, 5)    16384       input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_83 (BatchNo (None, 256, 5, 5)    20          conv2d_95[0][0]                  
__________________________________________________________________________________________________
activation_79 (Activation)      (None, 256, 5, 5)    0           batch_normalization_83[0][0]     
____________________________________________________________________________________________

In [20]:

model_input = []

for _ in range(100):
    board_tensor = np.random.randint(0, 3, size=(7, 5, 5))
    model_input.append(board_tensor)

model_input = np.array(model_input) 
 

action_target = []
for _ in range (100):
    search_prob = np.random.randn(26)
    #search_prob_flat = search_prob.reshape(25,)
    action_target.append(search_prob)
    
action_target = np.array(action_target)    


value_target = np.random.rand(100)
value_target = np.array(value_target) 




In [24]:
#pickle the data
import pickle 
def storeData():
    start = time.time()
    # database 
    db = {} 
    db['model_input'] = model_input 
    db['action_target'] = action_target 
    db['value_target'] = value_target 
      
    # Its important to use binary mode 
    dbfile = open('examplePickle', 'ab') 
      
    # source, destination 
    pickle.dump(db, dbfile)                      
    dbfile.close() 
    finish = time.time()
    print("Write Time taken :", finish - start)

def loadData(): 
    start = time.time()
    # for reading also binary mode is important 
    dbfile = open('examplePickle', 'rb')      
    db = pickle.load(dbfile) 
    """
    for keys in db: 
        print(keys, '=>', db[keys]) 
    """    
    dbfile.close() 
    finish = time.time()
    print("Read Time taken :", finish - start)
 
storeData()
loadData()

Write Time taken : 0.0016620159149169922
Read Time taken : 0.6699378490447998


In [25]:
#use HDF5 to store and load
import h5py


class ExperienceBuffer:
    def __init__(self, model_input, action_target, value_target):
        self.model_input = model_input
        self.action_target = action_target
        self.value_target = value_target
        
    def serialize(self, h5file):
        h5file.create_group('experience')
        h5file['experience'].create_dataset('model_input', data=self.model_input)
        h5file['experience'].create_dataset('action_target', data=self.action_target)
        h5file['experience'].create_dataset('value_target', data=self.value_target)

    
    def load_experience(self, h5file):
        return ExperienceBuffer(model_input=np.array(h5file['experience']['model_input']),
                               action_target=np.array(h5file['experience']['action_target']),
                               value_target=np.array(h5file['experience']['value_target'])
                               )

start = time.time() 
with h5py.File('test.hdf5', 'w') as exp_outf:
    ExperienceBuffer(model_input, action_target, value_target).serialize(exp_outf)
    
finish = time.time()
print("Write Time taken :", finish - start)  


start = time.time() 
with h5py.File('test.hdf5', 'r') as exp_input:
    experience_buffer = ExperienceBuffer(model_input, action_target, value_target).load_experience(exp_input)

finish = time.time()
print("Read Time taken :", finish - start)     

Write Time taken : 0.004450082778930664
Read Time taken : 0.0031900405883789062


In [21]:
from keras.optimizers import SGD
model.compile(SGD(lr=0.01), loss=['categorical_crossentropy', 'mse'])

model_small.compile(SGD(lr=0.01), loss=['categorical_crossentropy', 'mse'])

In [22]:
import time
start = time.time()
model.fit(model_input, [action_target, value_target], batch_size=64, epochs=1)
finish = time.time()
print("Time taken : ", finish - start)

start = time.time()
model_small.fit(model_input, [action_target, value_target], batch_size=64, epochs=1)
finish = time.time()
print("Time taken : ", finish - start)



Epoch 1/1
100/100 [==============================] - 16s 163ms/step - loss: -0.4110 - dense_13_loss: -0.1312 - dense_15_loss: 0.3524
Time taken :  29.608965158462524
Epoch 1/1
100/100 [==============================] - 0s 3ms/step - loss: -0.6854 - dense_16_loss: -1.1725 - dense_18_loss: 0.2327
Time taken :  1.0178492069244385


In [23]:
X = model_input[0]
X = np.expand_dims(X, axis=0)
print(X.shape)
prediction = model.predict(X)
print(prediction)

print("*"*60)
prediction = model_small.predict(X)
print(prediction)

(1, 7, 5, 5)
[array([[2.4195621e-25, 5.3648117e-09, 2.6260358e-20, 4.3068961e-15,
        8.1979097e-22, 8.5954981e-22, 1.0562663e-03, 2.4053762e-21,
        1.2376740e-17, 6.3513980e-24, 8.3003954e-08, 7.2274036e-03,
        5.8706249e-08, 3.1680581e-23, 2.1157889e-17, 1.5692027e-05,
        1.0464021e-15, 2.1030299e-29, 6.4227615e-24, 3.0220496e-22,
        2.0394633e-09, 4.3845105e-11, 7.2552381e-32, 1.4187485e-21,
        9.9170047e-01, 2.0737347e-09]], dtype=float32), array([[-0.32969806]], dtype=float32)]
************************************************************
[array([[0.03244389, 0.03802307, 0.03164793, 0.03486004, 0.02799989,
        0.04904727, 0.02354091, 0.03404812, 0.0494498 , 0.03783073,
        0.02743352, 0.03711371, 0.03104599, 0.03473119, 0.04067161,
        0.0344206 , 0.03974237, 0.03426522, 0.04344942, 0.04863343,
        0.03926243, 0.05658075, 0.04706815, 0.05077148, 0.04211408,
        0.03380449]], dtype=float32), array([[0.15468478]], dtype=float32)]


In [29]:
index = np.argmax(prediction[0])
rows = int(index/5)
cols = index%5
print("Move : ", (rows, cols))
print("Win chance :", prediction[1])

Move :  (3, 4)
Win chance : [[0.5846347]]


In [30]:
import tensorflow as tf
filepath = 'model'
#save the model
model.save('model')


In [31]:
"""
import tensorflow as tf
#load the model
loaded_model = tf.keras.models.load_model('model')
"""

"\nimport tensorflow as tf\n#load the model\nloaded_model = tf.keras.models.load_model('model')\n"